In [0]:
from google.colab import drive

drive.mount('/content/gdrive', force_remount = True)

Mounted at /content/gdrive


### Library

In [0]:
!pip3 install torch torchvision
!pip3 install unidecode
import zipfile
import re
import unidecode
import os
import unidecode
import string
import random
import time, math

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchtext.vocab import GloVe

from torchvision import datasets,transforms
from torch.autograd import Variable

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
num_epochs = 10000
print_every = 100
plot_every = 10
chunk_len = 200
embedding_size = 150
hidden_size = 100
batch_size =1
num_layers = 1
lr = 0.002
all_characters = string.printable
n_characters = len(all_characters)
print(all_characters)
print('num_chars = ', n_characters)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

num_chars =  100


## 1. Get the Text Data

In [0]:
file_path = "./gdrive/My Drive/data/shakespeare.txt"
file = open(file_path).read()
file_len = len(file)
print("file_len = ", file_len)

file_len =  720896


## 2. Functions for text processing

### 1) Random Chunk

In [0]:
def random_chunk():
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len + 1
    return file[start_index:end_index]

random_chunk()

"or Clarence!\n\nGLOUCESTER:\nThis is the fruit of rashness! Mark'd you not\nHow that the guilty kindred of the queen\nLook'd pale when they did hear of Clarence' death?\nO, they did urge it still unto the ki"

###2) Character to tensor

In [0]:
def char_tensor(string):
  tensor = torch.zeros(len(string)).long()
  for c in range(len(string)):
    tensor[c] = all_characters.index(string[c])
  return Variable(tensor).cuda()

print(char_tensor("ABCdef"))

tensor([36, 37, 38, 13, 14, 15], device='cuda:0')


3) Chunk into Input&label

In [0]:
def random_training_set():
  chunk = random_chunk()
  inp = char_tensor(chunk[:-1])
  target = char_tensor(chunk[1:])
  return inp, target

##3. Model & Optimizer


### 1) Model

In [0]:
class RNN(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.embedding_size = embedding_size
        
        self.encoder = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.RNN(embedding_size,hidden_size,num_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
        
    
    def forward(self, input, hidden):
        out = self.encoder(input.view(1,-1))  #embedding 만들어주는 코드
        out,hidden = self.rnn(out,hidden)
        out = self.decoder(out.view(batch_size,-1))
        
        return out,hidden

    def init_hidden(self):
        hidden = Variable(torch.zeros(self.num_layers, batch_size, hidden_size)).cuda()
        return hidden
    
model = RNN(n_characters, embedding_size, hidden_size, n_characters, num_layers=2).cuda()

In [0]:
inp = char_tensor("A")
print(inp)
hidden = model.init_hidden()
print(hidden.size())

out,hidden = model(inp,hidden)
print(out.size())

tensor([36], device='cuda:0')
torch.Size([2, 1, 100])
torch.Size([1, 100])


In [0]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_func = nn.CrossEntropyLoss()

Training

In [0]:
def evaluate():
    start_str = "b"
    inp = char_tensor(start_str)
    hidden = model.init_hidden()
    x = inp

    # print(start_str,end="^^\n")
    print("결과값: \n")
    for i in range(200):
        output,hidden = model(x,hidden)
        output_dist = output.data.view(-1).div(0.8).exp()
        #_,top_i = torch.max(output_dist,dim=0)
        top_i = torch.multinomial(output_dist, 1)[0]
        predicted_char = all_characters[top_i]

        
        print(predicted_char,end="")

        x = char_tensor(predicted_char)

In [0]:
loss_list = [] 
for i in range(num_epochs):
    total = char_tensor(random_chunk())
    inp = total[:-1]
    label = total[1:]
    hidden = model.init_hidden()

    loss = 0
    optimizer.zero_grad()
    for j in range(chunk_len-1):
        x  = inp[j]
        y_ = label[j].view(-1)
        y,hidden = model(x,hidden)
        loss += loss_func(y,y_)

    loss.backward()
    optimizer.step()
    
    if i % 100 == 0 and i!=0:
        print("\n{}번째 loss는 {}\n".format(i, loss/chunk_len))
        print("\n============================================\n")
        evaluate()


100번째 loss는 1.9460173845291138



결과값: 

lase it.

CORIOLANUS:
He rootlme carver.

PARDAY:
The hear ansence.

BRUTUSS:
Mother.

BUCKINGHANUS:
Mary?

GLOUCY:
Let me, God Lor enticgegtisher; angers, I hath, his hear late with spare held manish
200번째 loss는 2.1781539916992188



결과값: 

low?

FOFUS:
The lides mine the lave and not hush shall on his enfore for is but thou goin their dase the To onady, be donces a
Monzen revind him had, to the love of off, your fate this let I shall he
300번째 loss는 1.8689435720443726



결과값: 

les to grainst he dost
Thand at him forseld go lived welliss to the minetnose his deed them weall of there it frill we you thee sunior the concotter:
Thou the eart.

YORK:
But and be his can shard dea
400번째 loss는 1.7792878150939941



결과값: 

le betervand; Ely say the capson the will the hour ricizen, Richell, of there's bathing my in thus scould's our tame: as sours, ands let is give and were to best minds king of the surged for you jodiu
500번째 loss는 1.582981824874878

